In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
import gc

In [3]:
pd.set_option("display.max_columns", 1000)

In [11]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *
from attributes_recs import *
from learn_to_rank import *
from pandarallel import pandarallel
from lightfm import LightFM
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
combined_features = load_combined(["rubrics.csv","features.csv"], orgs_df=orgs_df)
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df))

16967 1252801


In [6]:
reviews = load_reviews("reviews.csv", users_df=users_df, orgs_df=orgs_df)
orgs_df=enrich_orgs(orgs_df, reviews)
print(f"{len(reviews)}")
reviews.head()

3640835


,user_id,org_id,rating,ts,aspects,good,user_city,in_test,org_city,travel
0,16998268288908323644,7184895086928047809,2.0,105,[],0,msk,0,msk,0
1,3121447338909258868,7184895086928047809,5.0,464,[],1,msk,0,msk,0
2,1970649778250883025,7184895086928047809,3.0,789,[],0,msk,0,msk,0
3,7554889464530643866,7184895086928047809,4.0,936,[],1,msk,0,msk,0
4,15907910894057053620,7184895086928047809,1.0,1143,[],0,msk,0,msk,0


In [7]:
test_users_df = test_users_with_city\
    .merge(reviews[reviews.good>0][['user_id','org_id']], on='user_id', how='left', indicator=True)
good_test_users = test_users_df[test_users_df["_merge"]=='both']\
    .groupby(['user_id','city'])\
    .agg(org_id=pd.NamedAgg("org_id", aggfunc=list)).reset_index()

empty_test_users = test_users_df[test_users_df["_merge"]=='left_only'][['user_id','city']]
empty_test_users['org_id'] = [[] for _ in empty_test_users.itertuples()]
final_test_users_df = pd.concat([good_test_users, empty_test_users], ignore_index=True)
del test_users_df
del good_test_users
del empty_test_users
print(len(final_test_users_df), len(test_users_with_city))

16967 16967


In [8]:
draft_train_reviews, draft_test_reviews = train_test_split(reviews, min_ts=1000)

In [9]:
train_pairs = draft_train_reviews.explode("org_id").drop(columns='city')
test_pairs = draft_test_reviews.explode("target").rename(columns={'target':'org_id'})

In [14]:
result, encs = prepare_reviews_lightfm(reviews, orgs_df,
                                   train_pairs, test_pairs,
                                   min_reviews_per_user=2,
                                   min_org_reviews=5,
                                   min_travels_reviews=1,
                                   min_org_score=3.5)
m = reviews_matrix(result, encs)

In [21]:
test_m = reviews_matrix(test_pairs, encs)

ValueError: row, column, and data array must all be the same length

In [15]:
model = LightFM(no_components=50)
model.fit(m, epochs=20)

In [16]:
model.predict(test_pairs['user_id'],test_pairs['org_'])

TypeError: predict() missing 1 required positional argument: 'item_ids'